In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

In [ ]:
# Load the Excel file
file_path = r'C:\Users\DELL\Supply_Chain_Case_Study.xlsb'
xlsb = pd.ExcelFile(file_path)

# Load the sheets individually
sales_df = pd.read_excel(xlsb, sheet_name='Sales')
demand_df = pd.read_excel(xlsb, sheet_name='Demand')
inventory_df = pd.read_excel(xlsb, sheet_name='Inventory')
product_master_df = pd.read_excel(xlsb, sheet_name='Product master')

In [6]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [7]:
sales_df.head()

,Region,Customer Name,Customer Ship Country,Product,Plant,Plant Country,Fiscal Week,Sales Amount,Sales-Quantity,No. of Delivery,Ontime Delivery,COGS
0,America,ABC Corp,US,2F0001-1,958,US,1,NaN,NaN,NaN,NaN,NaN
1,America,ABC Corp,US,2F0001-1,958,US,2,"1,358",4500.00,1.00,1.00,282.00
2,America,ABC Corp,US,2F0001-1,958,US,3,"4,840",18500.00,4.00,4.00,1537.00
3,America,ABC Corp,US,2F0001-1,958,US,4,"6,385",25000.00,3.00,3.00,3125.00
4,America,ABC Corp,US,2F0001-1,958,US,5,NaN,NaN,NaN,NaN,NaN


In [8]:
demand_df.head()

,Region,Customer Name,Customer Ship Country,Product,Plant Code,Plant Country,Fiscal Week,Future Demand-Amount,Future Demand-Quantity,Final Demand-Amount,Final Demand-Quantity,Demand 1 Weeks Before-Amount,Demand 1 Weeks Before-Quantity,Demand 4 Weeks Before-Amount,Demand 4 Weeks Before-Quantity,Lates-Amount,Lates-Quantity
0,America,ABC Corp,US,2F0001-1,958,US,1,NaN,NaN,NaN,NaN,NaN,NaN,2717.00,9000.00,NaN,NaN
1,America,ABC Corp,US,2F0001-1,958,US,2,NaN,NaN,1358.00,4500.00,1358.00,4500.00,3625.00,14000.00,NaN,NaN
2,America,ABC Corp,US,2F0001-1,958,US,3,NaN,NaN,4840.00,18500.00,3625.00,14000.00,4534.00,19000.00,NaN,NaN
3,America,ABC Corp,US,2F0001-1,958,US,4,NaN,NaN,5477.00,20000.00,908.00,5000.00,6342.00,23000.00,NaN,NaN
4,America,ABC Corp,US,2F0001-1,958,US,5,NaN,NaN,908.00,5000.00,908.00,5000.00,NaN,NaN,NaN,NaN


In [9]:
inventory_df.head()

,Region,Product,Plant,Plant Country,Fiscal Week,Inventory-Amount,Inventory-Quantity,Inventory Weight KG,Inventory Volume CM3
0,America,2B0001-1,1001,US,18,10900.00,1008.00,NaN,
1,America,2B0001-1,1001,US,19,NaN,NaN,NaN,
2,America,2B0001-1,1001,US,20,NaN,NaN,NaN,
3,America,2B0001-1,1001,US,21,1911.00,1680.00,NaN,
4,America,2B0001-1,1001,US,22,1911.00,1680.00,NaN,


In [10]:
product_master_df.head()

,Region,Product,Product Sub Category,Product Category,APQ
0,America,2F0001-1,ICON,Mobile,5000.00
1,America,000001-1,TV,Home Appliance,11692.00
2,America,010001-1,Refrigerator,Home Appliance,600.00
3,America,020001-1,Mixer,Home Appliance,264.00
4,America,020001-1,Refrigerator,Home Appliance,1600.00


In [11]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220097 entries, 0 to 220096
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Region                 220097 non-null  object 
 1   Customer Name          220097 non-null  object 
 2   Customer Ship Country  220097 non-null  object 
 3   Product                220097 non-null  object 
 4   Plant                  220097 non-null  int64  
 5   Plant Country          219924 non-null  object 
 6   Fiscal Week            220097 non-null  int64  
 7   Sales Amount           125428 non-null  object 
 8   Sales-Quantity         125428 non-null  float64
 9   No. of Delivery        124851 non-null  float64
 10  Ontime Delivery        124851 non-null  float64
 11  COGS                   125428 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 20.2+ MB


In [12]:
# Convert to numeric with coercion
sales_df['Sales Amount'] = pd.to_numeric(sales_df['Sales Amount'], errors='coerce')

# 1) Executive summary and trend of each KPI - Sales, Customer Demand, Lates and Inventory

In [13]:
# Group sales data by fiscal week and summarize sales amount and quantity
sales_summary = sales_df.groupby('Fiscal Week').agg(
    total_sales_amount=('Sales Amount', 'sum'),
    total_sales_quantity=('Sales-Quantity', 'sum'),
    total_deliveries=('No. of Delivery ', 'sum'),
    ontime_deliveries=('Ontime Delivery', 'sum')
).reset_index()
sales_summary.head()

,Fiscal Week,total_sales_amount,total_sales_quantity,total_deliveries,ontime_deliveries
0,1,364790.00,959087751.00,41234.00,37520.00
1,2,362419.00,1053114411.00,53020.00,48621.00
2,3,366926.00,943951244.00,45409.00,41466.00
3,4,351712.00,890391362.00,41918.00,37745.00
4,5,354224.00,1011587573.00,48673.00,45314.00


In [14]:
sum_of_total_sales_amount = sum(sales_summary['total_sales_amount'])
print(f'sum_of_total_sales_amount: {sum_of_total_sales_amount}')

sum_of_total_sales_amount: 14464511.0


In [15]:
# Time series plot for total_sales_amount vs fiscal week
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=sales_summary['Fiscal Week'], 
                          y=sales_summary['total_sales_amount'], 
                          mode='lines+markers',
                          name='Total Sales Amount'))
# Dark mode layout
fig1.update_layout(
    title='Total Sales Amount Over Fiscal Weeks',
    xaxis_title='Fiscal Week',
    yaxis_title='Total Sales Amount',
    template='plotly_dark',
    plot_bgcolor='#1e1e1e',  
    paper_bgcolor='#1e1e1e' )
fig1.show()

In [16]:
# Time series plot for total_sales_quantity vs fiscal week
fig = go.Figure()
fig.add_trace(go.Scatter(x=sales_summary['Fiscal Week'], 
                         y=sales_summary['total_sales_quantity'], 
                         mode='lines+markers',
                         name='Total Sales Quantity'))

# Add dark mode layout
fig.update_layout(
    title='Total Sales Quantity Over Fiscal Weeks',
    xaxis_title='Fiscal Week',
    yaxis_title='Total Sales Quantity',
    template='plotly_dark', 
    plot_bgcolor='#1e1e1e',  
    paper_bgcolor='#1e1e1e'  )
fig.show()


In [17]:
fig = go.Figure()

# Add first trace for total_sales_quantity
fig.add_trace(
    go.Scatter(
        x=sales_summary['Fiscal Week'], 
        y=sales_summary['total_sales_quantity'], 
        mode='lines+markers',
        name='Total Sales Quantity',
        line=dict(color='#17becf', width=2),
        marker=dict(color='#17becf', size=7),
        yaxis='y1'
    )
)

# Add second trace for total_sales_amount on the right axis
fig.add_trace(
    go.Scatter(
        x=sales_summary['Fiscal Week'], 
        y=sales_summary['total_sales_amount'], 
        mode='lines+markers',
        name='Total Sales Amount',
        line=dict(color='#ff6347', width=2, dash='dash'),
        marker=dict(color='#ff4500', size=7),
        yaxis='y2'
    )
)

# Dark mode layout with dual y-axes
fig.update_layout(
    title=dict(
        text='Total Sales Quantity and Amount Over Fiscal Weeks',
        font=dict(size=22, color='white')
    ),
    xaxis=dict(
        title='Fiscal Week',
        titlefont=dict(color='white'),
        tickfont=dict(color='white')
    ),
    yaxis=dict(
        title='Total Sales Quantity',
        titlefont=dict(color='#17becf'),
        tickfont=dict(color='#17becf')
    ),
    yaxis2=dict(
        title='Total Sales Amount',
        titlefont=dict(color='#ff6347'),
        tickfont=dict(color='#ff6347'),
        anchor='x',
        overlaying='y',
        side='right'
    ),
    template='plotly_dark',
    plot_bgcolor='#1e1e1e',  
    paper_bgcolor='#1e1e1e',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1,
        font=dict(color='white')
    ))
fig.show()


In [18]:
# Time series plot for total_deliveries and ontime_deliveries vs fiscal week
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=sales_summary['Fiscal Week'], 
                          y=sales_summary['total_deliveries'], 
                          mode='lines+markers',
                          name='Total Deliveries'))

fig3.add_trace(go.Scatter(x=sales_summary['Fiscal Week'], 
                          y=sales_summary['ontime_deliveries'], 
                          mode='lines+markers',
                          name='Ontime Deliveries'))

# Dark mode layout
fig3.update_layout(
    title='Total Deliveries vs Ontime Deliveries Over Fiscal Weeks',
    xaxis_title='Fiscal Week',
    yaxis_title='Number of Deliveries',
    template='plotly_dark', 
    plot_bgcolor='#1e1e1e', 
    paper_bgcolor='#1e1e1e')
fig3.show()


In [19]:
# Group demand data by fiscal week and summarize future demand, final demand
demand_summary = demand_df.groupby('Fiscal Week').agg(
    total_future_demand_amount=('Future Demand-Amount', 'sum'),
    total_final_demand_amount=('Final Demand-Amount', 'sum'),
    total_future_demand_quantity=('Future Demand-Quantity', 'sum'),
    total_final_demand_quantity=('Final Demand-Quantity', 'sum')
).reset_index()
demand_summary.head()

,Fiscal Week,total_future_demand_amount,total_final_demand_amount,total_future_demand_quantity,total_final_demand_quantity
0,1,0.00,51663823.00,0.00,983458938.00
1,2,0.00,57400497.00,0.00,1037890031.00
2,3,0.00,49970507.00,0.00,906398696.00
3,4,0.00,48186399.00,0.00,917221461.00
4,5,0.00,53758786.00,0.00,939941016.00


In [20]:
sum_of_total_future_demand_amount = sum(demand_summary['total_future_demand_amount'])
print(f'sum_of_total_future_demand_amount: {sum_of_total_future_demand_amount}')

sum_of_total_future_demand_amount: 477621826.0


In [21]:
# Plot total_future_demand_amount and total_future_demand_quantity
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=demand_summary['Fiscal Week'], y=demand_summary['total_future_demand_amount'],
                         mode='lines+markers', name='Future Demand Amount', line=dict(color='cyan')))
fig1.add_trace(go.Scatter(x=demand_summary['Fiscal Week'], y=demand_summary['total_future_demand_quantity'],
                         mode='lines+markers', name='Future Demand Quantity', line=dict(color='magenta')))
fig1.update_layout(title='Future Demand Amount and Quantity over Fiscal Week',
                   xaxis_title='Fiscal Week', yaxis_title='Amount / Quantity',
                   template='plotly_dark')
fig1.show()

In [22]:
fig2 = go.Figure()

# Add trace for Final Demand Quantity on primary y-axis
fig2.add_trace(go.Scatter(x=demand_summary['Fiscal Week'], y=demand_summary['total_final_demand_quantity'],
                         mode='lines+markers', name='Final Demand Quantity', line=dict(color='yellow')))

# Add trace for Final Demand Amount on secondary y-axis
fig2.add_trace(go.Scatter(x=demand_summary['Fiscal Week'], y=demand_summary['total_final_demand_amount'],
                         mode='lines+markers', name='Final Demand Amount', line=dict(color='cyan'),
                         yaxis='y2'))

# Update layout to include secondary y-axis
fig2.update_layout(
    title='Final Demand Amount and Quantity over Fiscal Week',
    xaxis_title='Fiscal Week',
    yaxis_title='Final Demand Quantity',
    yaxis2=dict(
        title='Final Demand Amount',
        overlaying='y',
        side='right'
    ),
    template='plotly_dark'
)
fig2.show()


In [23]:
# Group Inventory data by fiscal week and summarize inventory amount and quantity
inventory_summary = inventory_df.groupby('Fiscal Week').agg(
    total_inventory_amount=('Inventory-Amount', 'sum'),
    total_inventory_quantity=('Inventory-Quantity', 'sum')
).reset_index()
inventory_summary.head()

,Fiscal Week,total_inventory_amount,total_inventory_quantity
0,1,600229309.91,16497135239.06
1,2,599395114.32,16284576410.00
2,3,607112689.33,16605755953.00
3,4,608669852.38,16484520769.00
4,5,615502340.93,16763302815.00


In [24]:
sum_of_total_inventory_amount = sum(inventory_summary['total_inventory_amount'])
print(f'sum_of_total_inventory_amount: {sum_of_total_inventory_amount}')

sum_of_total_inventory_amount: 25838469341.84806


In [25]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=inventory_summary['Fiscal Week'], y=inventory_summary['total_inventory_amount'],
                         mode='lines+markers', name='Inventory Amount', line=dict(color='cyan')))

fig.add_trace(go.Scatter(x=inventory_summary['Fiscal Week'], y=inventory_summary['total_inventory_quantity'],
                         mode='lines+markers', name='Inventory Quantity', line=dict(color='magenta')))

# Update layout to dark mode
fig.update_layout(title='Inventory Amount and Quantity over Fiscal Week',
                  xaxis_title='Fiscal Week', yaxis_title='Amount / Quantity',
                  template='plotly_dark')
fig.show()

In [26]:
# Group sales data by fiscal week and summarize sales amount and quantity
sales_summary = sales_df.groupby('Fiscal Week').agg(
    total_sales_amount=('Sales Amount', 'sum'),
    total_sales_quantity=('Sales-Quantity', 'sum'),
    total_deliveries=('No. of Delivery ', 'sum'),
    ontime_deliveries=('Ontime Delivery', 'sum')
).reset_index()

# Group demand data by fiscal week and summarize future demand, final demand
demand_summary = demand_df.groupby('Fiscal Week').agg(
    total_future_demand_amount=('Future Demand-Amount', 'sum'),
    total_final_demand_amount=('Final Demand-Amount', 'sum'),
    total_future_demand_quantity=('Future Demand-Quantity', 'sum'),
    total_final_demand_quantity=('Final Demand-Quantity', 'sum')
).reset_index()

# Group Inventory data by fiscal week and summarize inventory amount and quantity
inventory_summary = inventory_df.groupby('Fiscal Week').agg(
    total_inventory_amount=('Inventory-Amount', 'sum'),
    total_inventory_quantity=('Inventory-Quantity', 'sum')
).reset_index()
inventory_summary.head()

# Merge the sales, demand and inventory summary to get a complete overview
# First merge sales_summary and demand_summary
temp_df = pd.merge(sales_summary, demand_summary, on='Fiscal Week', how='left')
# Then merge the result with inventory_summary
summary_df = pd.merge(temp_df, inventory_summary, on='Fiscal Week', how='left')
summary_df.head(10)
# Summary_df shows the sales_summary, demand_summary and inventory_summary against fiscal week in a single dataframe.

,Fiscal Week,total_sales_amount,total_sales_quantity,total_deliveries,ontime_deliveries,total_future_demand_amount,total_final_demand_amount,total_future_demand_quantity,total_final_demand_quantity,total_inventory_amount,total_inventory_quantity
0,1,364790.00,959087751.00,41234.00,37520.00,0.00,51663823.00,0.00,983458938.00,600229309.91,16497135239.06
1,2,362419.00,1053114411.00,53020.00,48621.00,0.00,57400497.00,0.00,1037890031.00,599395114.32,16284576410.00
2,3,366926.00,943951244.00,45409.00,41466.00,0.00,49970507.00,0.00,906398696.00,607112689.33,16605755953.00
3,4,351712.00,890391362.00,41918.00,37745.00,0.00,48186399.00,0.00,917221461.00,608669852.38,16484520769.00
4,5,354224.00,1011587573.00,48673.00,45314.00,0.00,53758786.00,0.00,939941016.00,615502340.93,16763302815.00
5,6,357031.00,936753015.00,44210.00,40622.00,0.00,53061720.00,0.00,967425218.00,617660312.15,16897112705.50
6,7,372904.00,843117580.00,44309.00,40676.00,0.00,49748287.00,0.00,869801835.00,621170182.18,16850789990.00
7,8,328777.00,855030774.00,38812.00,35668.00,0.00,42889979.00,0.00,854483656.00,627188724.24,16846372288.00
8,9,362335.00,996792211.00,45282.00,41838.00,0.00,51039810.00,0.00,973823811.00,623217141.42,16616042892.50
9,10,356186.00,916671979.00,42213.00,38566.00,0.00,48790533.00,0.00,905229519.00,631793022.77,16818554651.50


Summary_df shows the sales_summary, demand_summary and inventory_summary against fiscal week in a single dataframe.

## 2) Forecast accuracy of each customers [Compare demand 4 weeks before vs. Final Demand and 1 week before  vs. Final demand]

In [27]:
# Calculate forecast accuracy for 1 week and 4 weeks before
demand_df['Accuracy_1_week'] = (demand_df['Final Demand-Quantity'] - demand_df['Demand 1 Weeks Before-Quantity']).abs()
demand_df['Accuracy_4_weeks'] = (demand_df['Final Demand-Quantity'] - demand_df['Demand 4 Weeks Before-Quantity']).abs()

# Group by customer to find the average forecast accuracy
forecast_accuracy = demand_df.groupby('Customer Name').agg(
    avg_accuracy_1_week=('Accuracy_1_week', 'mean'),
    avg_accuracy_4_weeks=('Accuracy_4_weeks', 'mean')).reset_index()

forecast_accuracy

,Customer Name,avg_accuracy_1_week,avg_accuracy_4_weeks
0,ABC Corp,98089.92,93721.19
1,Corp de Limited,51727.58,71647.98
2,DRA Group,92465.76,121190.25
3,K S Company,88298.23,100811.34
4,LAG Public Firm,141146.76,157218.38
5,RBG Holding,217344.77,226243.58
6,SG Global Ventures,43546.54,66965.65
7,Sen Co Ltd,528792.38,575647.59
8,WMB Ltd.,1435.41,3304.78
9,Yay Partners,88375.81,119421.10


##### Forecast accuracy is about understanding how precise forecasts are on average, rather than just aggregating all forecast errors. 

In [28]:
# Plot the forecast accuracy for 1-week and 4-weeks forecasts
fig = px.bar(
    forecast_accuracy, 
    x='Customer Name', 
    y=['avg_accuracy_1_week', 'avg_accuracy_4_weeks'], 
    barmode='group', 
    title='Forecast Accuracy: 1 Week vs 4 Weeks Before',
    labels={'value': 'Forecast Accuracy (Mean Error)', 'Customer Name': 'Customer'},
    template='plotly_dark' )

fig.update_layout(
    xaxis_title='Customer Name',
    yaxis_title='Average Forecast Accuracy (Absolute Error)',
    legend_title='Forecast Type',
    width=800, height=500)

fig.show()

## 3) Percentage of shipments delivered on time compared to total shipment for all customers and plants.

In [29]:
# Calculate percentage of on-time deliveries
sales_df['Ontime_Delivery_Percentage'] = (sales_df['Ontime Delivery'] / sales_df['No. of Delivery ']) * 100

# Group by customer and plant to find the average on-time percentage
ontime_percentage = sales_df.groupby(['Customer Name', 'Plant']).agg(
    avg_ontime_delivery=('Ontime_Delivery_Percentage', 'mean')
).reset_index()

ontime_percentage.head(10)

,Customer Name,Plant,avg_ontime_delivery
0,ABC Corp,0,NaN
1,ABC Corp,494,76.36
2,ABC Corp,495,52.30
3,ABC Corp,496,66.67
4,ABC Corp,498,50.00
5,ABC Corp,608,0.00
6,ABC Corp,737,38.32
7,ABC Corp,747,99.55
8,ABC Corp,777,28.33
9,ABC Corp,812,100.00


In [30]:
ontime_percentage['Plant'] = ontime_percentage['Plant'].astype(str)
ontime_percentage = ontime_percentage.sort_values(by=['Customer Name','avg_ontime_delivery'], ascending = [True, False])

In [31]:
colors = [
    '#636EFA',  # Blue
    '#EF553B',  # Red
    '#00CC96',  # Green
    '#AB63FA',  # Purple
    '#FFA15A',  # Orange
    '#19D3F3',  # Light Blue
    '#FF6692',  # Pink
    '#B6E880',  # Light Green
    '#FF97FF'   # Magenta
]

# Loop through each customer and create individual plots
unique_customers = ontime_percentage['Customer Name'].unique()  # Get all unique customer names

for idx, customer in enumerate(unique_customers):
    # Filter data for the specific customer
    customer_data = ontime_percentage[ontime_percentage['Customer Name'] == customer]

    # Create a bar plot for the customer with a specific color
    fig = px.bar(
        customer_data, 
        x='Plant', 
        y='avg_ontime_delivery', 
        title=f'On-Time Delivery Percentage for {customer}',
        labels={'avg_ontime_delivery': 'Avg. On-Time Delivery (%)'},
        template='plotly_dark',  # Dark mode theme
        height=400, width=800
    )

    # Update traces to use the same color for all bars in this plot
    fig.update_traces(marker_color=colors[idx % len(colors)]) 

    # Customize layout and axis
    fig.update_layout(
        xaxis_title='Plant',
        yaxis_title='On-Time Delivery (%)',
        legend_title='Plant',
        yaxis=dict(range=[0, 100]), 
        title_x=0.5 
    )
    # Rotate x-axis labels for better readability
    fig.update_xaxes(tickangle=45)
    fig.show()


##  4) Identify products where excess inventory is available based on future demand and demand fluctuations.

In [32]:
# Group inventory data by product to summarize stock levels
inventory_summary_by_poduct = inventory_df.groupby('Product').agg(
    total_inventory=('Inventory-Quantity', 'sum')
).reset_index()

# Group demand data by product to summarize future demand
demand_summary_by_product = demand_df.groupby('Product').agg(
    total_future_demand_quantity=('Future Demand-Quantity', 'sum')
).reset_index()

# Compare inventory to future demand
excess_inventory = pd.merge(inventory_summary_by_poduct, demand_summary_by_product, on='Product', how='left')
excess_inventory['excess_stock'] = excess_inventory['total_inventory'] - excess_inventory['total_future_demand_quantity']

# Filter out products with excess stock
excess_inventory = excess_inventory[excess_inventory['excess_stock'] > 0]

# Display products with excess inventory
excess_inventory.head(10)

,Product,total_inventory,total_future_demand_quantity,excess_stock
0,000001-1,38144541.00,24000.00,38120541.00
1,010001-1,1012211790.00,3929580.00,1008282210.00
2,020001-1,2148476496.00,32116416.00,2116360080.00
3,030001-1,5182102310.00,173304000.00,5008798310.00
4,040001-1,19579937154.00,275554443.00,19304382711.00
5,050001-1,3035904191.00,85120617.00,2950783574.00
6,060001-1,4663329390.00,12467382.00,4650862008.00
7,070001-1,25363560766.00,175939541.00,25187621225.00
8,080001-1,26182112336.00,760740404.00,25421371932.00
9,090001-1,576862280.00,6968081.00,569894199.00


In [33]:
fig = px.bar(
    excess_inventory, 
    x='excess_stock', 
    y='Product', 
    orientation='h', 
    title='Excess Stock by Product', 
    labels={'excess_stock': 'Excess Stock Quantity', 'Product': 'Product'},
    color='excess_stock',  
    color_continuous_scale='Viridis', 
    template='plotly_dark',  
    height=1500, 
    width=900)
# Step 2: Customize layout for better readability
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'}, 
    xaxis_title='Excess Stock Quantity',
    yaxis_title='Product',
    margin=dict(t=50, l=100, r=50, b=50),
    title_x=0.5 
)
fig.show()


## 5) Average Week of Stock each plant is holding compared to future demand.

In [34]:
demand_df = demand_df.rename(columns={'Plant Code':'Plant'})

In [35]:
# Group inventory by plant to calculate average stock level
plant_inventory = inventory_df.groupby('Plant').agg(
    avg_stock=('Inventory-Quantity', 'mean')
).reset_index()

# Calculate the average weeks of stock by comparing inventory to future demand
plant_demand = demand_df.groupby('Plant').agg(
    avg_future_demand=('Future Demand-Quantity', 'mean')
).reset_index()

# Merge to compare stock levels to demand
weeks_of_stock = pd.merge(plant_inventory, plant_demand, left_on='Plant', right_on='Plant')
weeks_of_stock['weeks_of_stock'] = weeks_of_stock['avg_stock'] / weeks_of_stock['avg_future_demand']
weeks_of_stock.head(10)

,Plant,avg_stock,avg_future_demand,weeks_of_stock
0,0,NaN,1800.00,NaN
1,111,12673370.15,NaN,NaN
2,115,NaN,NaN,NaN
3,310,NaN,NaN,NaN
4,488,5970564.85,9000.00,663.40
5,493,-37063.78,NaN,NaN
6,494,206982.72,950.05,217.87
7,495,13590928.14,416253.10,32.65
8,496,922816.04,24721.45,37.33
9,498,249155.44,13286.08,18.75


In [36]:
weeks_of_stock['Plant'] = weeks_of_stock['Plant'].astype(str)

In [37]:
# Identify the specific plants with large stock values
specific_plants = weeks_of_stock[weeks_of_stock['Plant'].isin(['1294', '1043'])]
other_plants = weeks_of_stock[~weeks_of_stock['Plant'].isin(['1294', '1043'])]

# Create a horizontal bar chart for other plants
fig_others = px.bar(
    other_plants, 
    x='weeks_of_stock', 
    y='Plant', 
    orientation='h',  
    title='Average Weeks of Stock by Plant (Excluding Plants 1294 and 1043)', 
    labels={'weeks_of_stock': 'Average Weeks of Stock', 'Plant': 'Plant'},
    color='weeks_of_stock',  
    color_continuous_scale='Cividis', 
    template='plotly_dark', 
    height=600, 
    width=900
)

# Customize layout for better readability
fig_others.update_layout(
    yaxis={'categoryorder': 'total ascending'}, 
    xaxis_title='Average Weeks of Stock',
    yaxis_title='Plant',
    margin=dict(t=50, l=100, r=50, b=50), 
    title_x=0.5)
fig_others.show()

# Create a separate horizontal bar chart for plant 1294
fig_specific_1294 = px.bar(
    specific_plants[specific_plants['Plant'] == '1294'], 
    x='weeks_of_stock', 
    y='Plant', 
    orientation='h',
    title='Average Weeks of Stock for Plant 1294', 
    labels={'weeks_of_stock': 'Average Weeks of Stock', 'Plant': 'Plant'},
    color='weeks_of_stock',  
    color_continuous_scale='Viridis', 
    template='plotly_dark',  
    height=400,
    width=900)

# Customize layout for better readability
fig_specific_1294.update_layout(
    xaxis_title='Average Weeks of Stock',
    yaxis_title='Plant',
    margin=dict(t=50, l=100, r=50, b=50), 
    title_x=0.5)
fig_specific_1294.show()

# Create a separate horizontal bar chart for plant 1043
fig_specific_1043 = px.bar(
    specific_plants[specific_plants['Plant'] == '1043'], 
    x='weeks_of_stock', 
    y='Plant', 
    orientation='h',  
    title='Average Weeks of Stock for Plant 1043', 
    labels={'weeks_of_stock': 'Average Weeks of Stock', 'Plant': 'Plant'},
    color='weeks_of_stock', 
    color_continuous_scale='Plasma', 
    template='plotly_dark', 
    height=400,
    width=900)

# Customize layout for better readability
fig_specific_1043.update_layout(
    xaxis_title='Average Weeks of Stock',
    yaxis_title='Plant',
    margin=dict(t=50, l=100, r=50, b=50), 
    title_x=0.5)
fig_specific_1043.show()


Plant 1294 and 1043 has very high average weeks of stocks when compared to other plants, so plotting these two separately for better visibility and understanding.

## 6) Predict Potential Sales based on open demand and forecast accuracy.

In [38]:
# Predict sales using final demand and accuracy
demand_df['predicted_sales'] = demand_df['Final Demand-Quantity'] * (1 - (demand_df['Accuracy_1_week'] / demand_df['Final Demand-Quantity']))

# Summarize potential sales by customer
potential_sales = demand_df.groupby('Customer Name').agg(
    total_predicted_sales=('predicted_sales', 'sum')
).reset_index()
potential_sales.head(10)


,Customer Name,total_predicted_sales
0,ABC Corp,3554420134.00
1,Corp de Limited,1932033330.00
2,DRA Group,3673671491.00
3,K S Company,4316426071.00
4,LAG Public Firm,2712254130.00
5,RBG Holding,798925108.00
6,SG Global Ventures,2133128145.00
7,Sen Co Ltd,1931629661.00
8,WMB Ltd.,8473090.00
9,Yay Partners,3326493948.00


Above code calculates the predicted sales for each customer based on how closely the previous forecasts matched actual demand. It uses this adjusted figure to provide a summarized view of potential sales per customer, which can be valuable for forecasting and inventory planning.

In [39]:
potential_sales = potential_sales.sort_values(by='total_predicted_sales', ascending=False)

In [40]:
# Create a bar plot for total predicted sales by customer
fig = px.bar(
    potential_sales, 
    x='Customer Name', 
    y='total_predicted_sales', 
    title='Total Predicted Sales by Customer', 
    labels={'total_predicted_sales': 'Total Predicted Sales', 'Customer Name': 'Customer'}, 
    color='total_predicted_sales', 
    color_continuous_scale='Viridis', 
    template='plotly_dark',  
    height=600,  
    width=900 )

# Customize layout for better readability
fig.update_layout(
    xaxis_title='Customer Name',
    yaxis_title='Total Predicted Sales',
    margin=dict(t=50, l=50, r=50, b=50),  
    title_x=0.5, 
    yaxis=dict(showgrid=True, gridcolor='gray') )
fig.show()


# Future Scope using Machine Learning Model

1) Customer Segmentation Using RFM Analysis (Recency, Frequency, Monetary)- Segment customers based on their purchasing behavior using Recency, Frequency, and Monetary values.

2) Product Sales Forecasting Using time series algorithms- to forecast future sales based on historical data.

3) Customer Churn Prediction Using Machine Learning- Identify customers likely to churn (stop buying) using classification algorithms.

4) Seasonality Detection in Demand- Detect seasonality patterns in customer demand using time series decomposition.